In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset_dir = '/content/drive/MyDrive/IndoorOutdoorDataset'

In [4]:
img_width, img_height = 224, 224
batch_size = 32
epochs = 100

In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='training',
)

Found 800 files belonging to 2 classes.
Using 640 files for training.


In [6]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='validation',
)

Found 800 files belonging to 2 classes.
Using 160 files for validation.


In [7]:
model = models.Sequential()
model.add(ResNet50V2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3), classes=2))
model.add(layers.Flatten())
model.add(layers.Dense(264, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

94668760/94668760 [==============================] - 3s 0us/step


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 264)               26493192  
                                                                 
 dense_1 (Dense)             (None, 128)               33920     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 50092041 (191.09 MB)
Trainable params: 50046601 (190.91 MB)
Non-trainable params: 45440 (177.50 KB)
_________________________________________________________________


In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds
)

Epoch 1/100
20/20 [==============================] - 149s 3s/step - loss: 1.5992 - accuracy: 0.8250 - val_loss: 87.0129 - val_accuracy: 0.5125
Epoch 2/100
20/20 [==============================] - 9s 413ms/step - loss: 0.3046 - accuracy: 0.9031 - val_loss: 532.5679 - val_accuracy: 0.6938
Epoch 3/100
20/20 [==============================] - 8s 333ms/step - loss: 0.4941 - accuracy: 0.8672 - val_loss: 918.5499 - val_accuracy: 0.5437
Epoch 4/100
20/20 [==============================] - 8s 347ms/step - loss: 0.5616 - accuracy: 0.8313 - val_loss: 1252675.3750 - val_accuracy: 0.5000
Epoch 5/100
20/20 [==============================] - 8s 336ms/step - loss: 0.6050 - accuracy: 0.8156 - val_loss: 580835328.0000 - val_accuracy: 0.4938
Epoch 6/100
20/20 [==============================] - 8s 335ms/step - loss: 0.4475 - accuracy: 0.8266 - val_loss: 6093164.5000 - val_accuracy: 0.4938
Epoch 7/100
20/20 [==============================] - 8s 343ms/step - loss: 0.5521 - accuracy: 0.8359 - val_loss: 6612.

In [11]:
model.save('/content/drive/MyDrive/resnet50v2sequence_indoor_outdoor_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
model.evaluate(val_ds)

5/5 [==============================] - 2s 100ms/step - loss: 0.3861 - accuracy: 0.8562


[0.3860969543457031, 0.856249988079071]